In [1]:
import pandas as pd 
import numpy as np 

# Import TF Functions
import sys 
sys.path.append(r'C:\Users\kurt-\Documents\GitHub\GNN_Soil_Dynamics\TF Functions')
from TTF import TTF

In [2]:
# Read all files with csv
import os
import re
import matplotlib.pyplot as plt
path_out = r"FLAC\FLAC2D\1D Soil_Bedrock\TF_HLC"
path_in = r"FLAC\FLAC2D\\1D Soil_Bedrock\HLC Results"


In [3]:
files = [f for f in os.listdir(path_in) if f.endswith('.csv')]
files

['Data_0.000_.csv',
 'Data_1.000_.csv',
 'Data_10.000_.csv',
 'Data_100.000_.csv',
 'Data_101.000_.csv',
 'Data_102.000_.csv',
 'Data_103.000_.csv',
 'Data_104.000_.csv',
 'Data_105.000_.csv',
 'Data_106.000_.csv',
 'Data_107.000_.csv',
 'Data_108.000_.csv',
 'Data_109.000_.csv',
 'Data_11.000_.csv',
 'Data_110.000_.csv',
 'Data_111.000_.csv',
 'Data_112.000_.csv',
 'Data_113.000_.csv',
 'Data_114.000_.csv',
 'Data_115.000_.csv',
 'Data_116.000_.csv',
 'Data_117.000_.csv',
 'Data_118.000_.csv',
 'Data_119.000_.csv',
 'Data_12.000_.csv',
 'Data_120.000_.csv',
 'Data_121.000_.csv',
 'Data_122.000_.csv',
 'Data_123.000_.csv',
 'Data_124.000_.csv',
 'Data_125.000_.csv',
 'Data_126.000_.csv',
 'Data_127.000_.csv',
 'Data_128.000_.csv',
 'Data_129.000_.csv',
 'Data_13.000_.csv',
 'Data_130.000_.csv',
 'Data_131.000_.csv',
 'Data_132.000_.csv',
 'Data_133.000_.csv',
 'Data_134.000_.csv',
 'Data_135.000_.csv',
 'Data_136.000_.csv',
 'Data_137.000_.csv',
 'Data_138.000_.csv',
 'Data_139.000_.cs

In [4]:
# Extract the index saved with the files
indices = []
for file in files:
    pattern = r'Data_(\d+(\.\d+)?)'
    match = re.search(pattern, file)
    if match:
        index = float(match.group(1))
        indices.append(int(index))

# Print the extracted indices
print(indices)

Vs = np.genfromtxt(r"FLAC\FLAC2D\1D Soil_Bedrock\model_arrays_HLC.csv", delimiter=',', skip_header=1, filling_values=np.nan)
print(Vs.shape)

Vs_arrays = Vs[indices]




[0, 1, 10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 11, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 12, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 13, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 14, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 15, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 16, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 17, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 19, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 2, 20, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 21, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 23, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 24, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 25, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 26, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 27, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 28, 280, 281, 282,

In [12]:
# Create a class that reads the csv and appends the TTF data
class TTF_Reader:
    def __init__(self, path_in, path_out):
        self.path_in = path_in
        self.path_out = path_out
        self.files = [ f for f in os.listdir(self.path_in) if f.endswith('.csv')]
        self.data = []
        
    def read(self):
        prev_freq = None
        save = 0
        for file in self.files:
            data = pd.read_csv(os.path.join(self.path_in, file))
            freq, TTF_val = TTF(data.iloc[:, 2].values, data.iloc[:, 1].values, data.iloc[0, 0], n_points=1000)
            # Check if the frequency is the same
            if prev_freq is not None:
                if np.allclose(prev_freq, freq) == False:
                    print(f"Frequency is not the same for file {file}")
                    break
                else:
                    prev_freq = freq
                    
            if save == 0:
                np.savetxt(os.path.join(self.path_out,f"TTF_freq.csv"), freq, delimiter=',')
                save = 1
            # Just append TTF
            self.data.append(TTF_val)
            print(f"File {file} has been read")
            
        return self.data

In [13]:
A = TTF_Reader(path_in, path_out)
A.read()


File Data_0.000_.csv has been read
File Data_1.000_.csv has been read
File Data_10.000_.csv has been read
File Data_100.000_.csv has been read
File Data_101.000_.csv has been read
File Data_102.000_.csv has been read
File Data_103.000_.csv has been read
File Data_104.000_.csv has been read
File Data_105.000_.csv has been read


KeyboardInterrupt: 

In [9]:
# Save the data to a pt file
import pickle

with open(os.path.join(path_out,'TTF_data_200.pt'), 'wb') as f:
    pickle.dump(A.data, f)
    print('Data has been saved')


Data has been saved


In [10]:

# Save the Vs values
with open(os.path.join(path_out,'Vs_values_200.pt'), 'wb') as f:
    pickle.dump(Vs_arrays, f)
    print('Vs values have been saved')

Vs values have been saved
